In [1]:
#TALLER

In [2]:
import pandas as pd
import numpy as np

In [3]:
#PUNTO 1

In [4]:
#Importando tambien los datos de la GEIH 2025, directamente desde el repositorio del profesor
df_cgsse = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/cgsse.CSV"
                       ,encoding="latin", sep= ";") 
df_ft = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"
                       ,encoding="latin", sep= ";", low_memory=False) 
df_ocupado = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
                       ,encoding="latin", sep= ";", low_memory=False) 
df_desocupados = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/No%20ocupados.CSV"
                       ,encoding="latin", sep= ";") 

In [5]:
# 1. Poblacion total de colombia: 
ptc = df_cgsse["FEX_C18"].sum()
print('Población total de colombia:', int(ptc))

Población total de colombia: 52045001


In [6]:
#Poblacion en edad de trabajar y en no edad de trabajar

sin_edad_trabajar = df_cgsse[df_cgsse['P6040']<15]['FEX_C18'].sum()
edad_trabajar = df_cgsse[df_cgsse['P6040']>=15]['FEX_C18']
total_edad_trabajar = df_cgsse[df_cgsse['P6040']>=15]['FEX_C18'].sum()

print('La poblacion total sin edad para trabajar es:',int(sin_edad_trabajar))
print('La poblacion total con edad para trabjar es:',int(total_edad_trabajar))


La poblacion total sin edad para trabajar es: 11391585
La poblacion total con edad para trabjar es: 40653416


In [7]:
#Calculado ahora la poblacion economicamente activa
pea = df_ft[df_ft['FT']==1]['FEX_C18'].sum()
print('La poblacion economicamente activa es:',int(pea))

La poblacion economicamente activa es: 25974846


In [8]:
#Calculo de Poblacion economicamente inactiva
pei = df_ft[df_ft['FFT']==1]['FEX_C18'].sum()
print('La poblacion economicamente inactiva es:',int(pei))


La poblacion economicamente inactiva es: 14678569


In [9]:
# Tasa de desempleo
desocupados = df_desocupados['FEX_C18'].sum()
td = (desocupados / pea) * 100
print('La tasa de desempleo es:', round(td, 2), '%')

La tasa de desempleo es: 65.06 %


In [10]:
#Cantiadad de personas desempleadas
D = df_desocupados[df_desocupados['DSI']==1]['FEX_C18'].sum()
print('La cantidad de personas desempleadas es:', int(D))

#Tasa de desempleados
td = (D / pea)
print(f'La tasa de desempleo es:{td*100} %')

La cantidad de personas desempleadas es: 2221893
La tasa de desempleo es:8.554020086155528 %


In [11]:
# Tasa de ocupados
O = df_ocupado['FEX_C18'].sum()
print('La cantidad de personas ocupadas es:', int(O))

#poblacion en edad de trabajar
pet = df_ft['FEX_C18'].sum()
print('La poblacion en edad de trabajar es:', int(pet))

#TASA DE OCUPADOS
to = (O / pet) 
print(f'La tasa de ocupados es: {to:.2%}')

La cantidad de personas ocupadas es: 23752952
La poblacion en edad de trabajar es: 40653416
La tasa de ocupados es: 58.43%


In [12]:
#EJERCICIO EN CLASE

In [13]:
#identificar el departamento
df_ft['DPTO'].unique()

array([ 8, 70, 41, 63, 66,  5, 11, 13, 15, 17, 18, 19, 20, 23, 25, 27, 44,
       47, 50, 52, 54, 68, 73, 76, 81, 85, 86, 88, 91, 94, 95, 97, 99])

In [14]:
df_desocupados = pd.merge(df_desocupados, df_cgsse[['DIRECTORIO', 'SECUENCIA_P', 'ORDEN', 'P3271']],
                        on=['DIRECTORIO', 'SECUENCIA_P', 'ORDEN'], how='left')

df_ft = pd.merge(df_ft, df_cgsse[['DIRECTORIO', 'SECUENCIA_P', 'ORDEN', 'P3271']],
                        on=['DIRECTORIO', 'SECUENCIA_P', 'ORDEN'], how='left')

In [15]:
# tasa de desempleo en Quindío Caldas y Risaralda
MDQ = df_desocupados[(df_desocupados['DSI']==1) & (df_desocupados['DPTO']==63) & (df_desocupados['P3271']==2)]['FEX_C18'].sum()
print(f'Mujeres Desempleadas en el Quindio: {MDQ}')
# 2. Calcular cantidad de mujeres en la PEA del quindío
PEAMQ = df_ft[(df_ft['FT']==1) & (df_ft['DPTO']==63) & (df_ft['P3271']==2) ]['FEX_C18'].sum()
print(f'Poblacion Economicamente Activa de mujeres en el quindío: {PEAMQ}')

# 3. Calcular la tasa de desempleo en mujeres del quindío
TDMQ = MDQ/PEAMQ
print(f'Tasa de Desempleo en mujeres del quindío: {TDMQ*100}%')

Mujeres Desempleadas en el Quindio: 10618.534945967001
Poblacion Economicamente Activa de mujeres en el quindío: 116885.879263648
Tasa de Desempleo en mujeres del quindío: 9.084531863781265%


In [26]:
PEAHQ = df_ft[(df_ft['FT']==1) & (df_ft['DPTO']==63) & (df_ft['P3271']==1) ]['FEX_C18'].sum()
print(f'PEAHQ:', int(PEAHQ))


PEAHQ: 174625


In [25]:
HDQ = df_desocupados[(df_desocupados['DSI']==1) & (df_desocupados['DPTO']==63) & (df_desocupados['P3271']==1)]['FEX_C18'].sum()
print(f'Hombres Desempleados en el Quindio: {HDQ}')
PEAHQ = df_ft[(df_ft['FT']==1) & (df_ft['DPTO']==63) & (df_ft['P3271']==1) ]['FEX_C18'].sum()
print(f'Poblacion Economicamente Activa de hombres en el quindío: {PEAHQ}')
TDHQ = HDQ/PEAHQ
print(f'Tasa de Desempleo en hombres del quindío:', int(TDHQ*100), '%')

Hombres Desempleados en el Quindio: 17869.520960767004
Poblacion Economicamente Activa de hombres en el quindío: 174625.87195169798
Tasa de Desempleo en hombres del quindío: 10 %


In [18]:
#RISARALDA

In [19]:
HDR = df_desocupados[(df_desocupados['DSI']==1) & (df_desocupados['DPTO']==66) & (df_desocupados['P3271']==1)]['FEX_C18'].sum()
print(f'Hombres Desempleados en el Risaralda: {HDR}')
PEAHR = df_ft[(df_ft['FT']==1) & (df_ft['DPTO']==66) & (df_ft['P3271']==1) ]['FEX_C18'].sum()
print(f'Poblacion Economicamente Activa de hombres en el Risaralda: {PEAHR}')
TDHR = HDR/PEAHR
print(f'Tasa de Desempleo en hombres del Risaralda: {TDHR*100}%')

Hombres Desempleados en el Risaralda: 18000.60370715
Poblacion Economicamente Activa de hombres en el Risaralda: 296342.328368219
Tasa de Desempleo en hombres del Risaralda: 6.074260064793518%


In [20]:
MDR = df_desocupados[(df_desocupados['DSI']==1) & (df_desocupados['DPTO']==66) & (df_desocupados['P3271']==2)]['FEX_C18'].sum()
print(f'Mujeres Desempleadas en el Risaralda: {MDR}')
PEAMR = df_ft[(df_ft['FT']==1) & (df_ft['DPTO']==66) & (df_ft['P3271']==2) ]['FEX_C18'].sum()
print(f'Poblacion Economicamente Activa de mujeres en el Risaralda: {PEAMR}')
TDMR = MDR/PEAMR
print(f'Tasa de Desempleo en mujeres del Risaralda: {TDMR*100}%')

Mujeres Desempleadas en el Risaralda: 21676.799766712
Poblacion Economicamente Activa de mujeres en el Risaralda: 203157.903399127
Tasa de Desempleo en mujeres del Risaralda: 10.669926891362644%


In [21]:
#CALDAS

In [22]:
MDC = df_desocupados[(df_desocupados['DSI']==1) & (df_desocupados['DPTO']==17) & (df_desocupados['P3271']==2)]['FEX_C18'].sum()
print(f"Cantidad de Mujeres Desempleadas en el Caldas: {MDC}")
PEAHC = df_ft[(df_ft['FT']==1) & (df_ft['DPTO']==17) & (df_ft['P3271']==2) ]['FEX_C18'].sum()
print(f'Poblacion Economicamente Activa de mujeres en el Caldas: {PEAHC}')
TDHC = MDC/PEAHC
print(f'Tasa de Desempleo en hombres del Caldas: {TDHC*100}%')

Cantidad de Mujeres Desempleadas en el Caldas: 17214.491449226003
Poblacion Economicamente Activa de mujeres en el Caldas: 224206.533775
Tasa de Desempleo en hombres del Caldas: 7.677961547053494%


In [23]:
HDC =MDC = df_desocupados[(df_desocupados['DSI']==1) & (df_desocupados['DPTO']==17) & (df_desocupados['P3271']==1)]['FEX_C18'].sum()
print(f"Cantidad de Mujeres Desempleadas en el Caldas: {MDC}")
PEAHC = df_ft[(df_ft['FT']==1) & (df_ft['DPTO']==17) & (df_ft['P3271']==1) ]['FEX_C18'].sum()
print(f'Poblacion Economicamente Activa de muejeres en el Caldas: {PEAHC}')
TDHC = MDC/PEAHC
print(f'Tasa de Desempleo en mujeres del Caldas: {TDHC*100}%')

Cantidad de Mujeres Desempleadas en el Caldas: 18226.160047418998
Poblacion Economicamente Activa de muejeres en el Caldas: 326447.467769983
Tasa de Desempleo en mujeres del Caldas: 5.5831831601957065%
